# Selenium 크롤러

In [5]:
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime

In [6]:
import os
os.listdir('../data/') # url 폴더에 어떤 파일이 있는지 확인

['R언급량test.csv',
 'R언급량test2.csv',
 'R엣지링크test.csv',
 'R엣지링크test2.csv',
 'stopwords.txt',
 '갓생.csv',
 '갓생_2022.csv',
 '갓생_content_합본.csv',
 '갓생_tokenized.csv']

In [12]:
file_name = '갓생_2022.csv'
url_list = pd.read_csv('../data/갓생_2022.csv', index_col=0)

In [13]:
url_list

,url
0,https://blog.naver.com/owogy/222609847793
1,https://blog.naver.com/sjksnow/222609841004
2,https://blog.naver.com/wlgml3742/222609846552
3,https://blog.naver.com/minyung1240/222609810405
4,https://blog.naver.com/doublestepthatswhy/2226...
...,...
254476,https://blog.naver.com/amoureuxzz/222971512674
254477,https://blog.naver.com/waterlight33/222971510182
254478,https://blog.naver.com/kong1264k/222971511604
254479,https://blog.naver.com/nahyun050405/222971512079


In [14]:
url_list = list(url_list['url'])

In [15]:
# 크롤링한 내용 담을 DataFrame
df = pd.DataFrame(columns=['title', 'nickname', 'content', 'date', 'tags'])

## Test 10개만 돌려보기

In [16]:
path = "chromedriver" # 웹드라이버 실행
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(path, chrome_options=options)
list_error = list()

# for i in tqdm_notebook(range(0, len(url_list))):
for i in tqdm(range(0, 10)):    
#     print(i)
    
    driver.get(url_list[i])
    
    
    try:
        driver.switch_to.frame("mainFrame")
        target_info = {}
        
        try: # 제목
            overlays = 'se-module.se-module-text.se-title-text'
            title = driver.find_element(By.CLASS_NAME, overlays).text
            
        except:
            title = ' '
            
        try: # 날짜
            overlays = 'se_publishDate.pcol2'
            date = driver.find_element(By.CLASS_NAME, overlays).text
#             date = datetime.datetime.strptime(day,'%Y-%m-%d')
            
        except:
            date = ' '
        
        try: # 글작성자
            overlays = 'link.pcol2'
            author = driver.find_element(By.CLASS_NAME, overlays).text
            
        except:
            author = ' '
            
        try: # 콘텐츠 (글 내용)
            overlays = "se-component.se-text.se-l-default"
            contents = driver.find_elements(By.CLASS_NAME, overlays)
            content_list = []
            
            for c in contents:
                content_list.append(c.text)
                
            content_str = ' '.join(content_list)
            
        except:
            content_str = ' '
            
        try: # 태그 크롤링
            overlays = "post_footer_contents" 
            tags = driver.find_element(By.CLASS_NAME, overlays).text
            
        except:
            tags = ' '
        
        target_info['title'] = title
        target_info['nickname'] = author
        target_info['content'] = content_str
        target_info['date'] = date
        target_info['tags'] = tags
    
        df1 = pd.DataFrame(target_info, index = [i])
        df = pd.concat([df, df1])
      
    except:
        list_error.append(i)
        continue

C:\Users\si\AppData\Local\Temp\ipykernel_11892\1517268150.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(path, chrome_options=options)


  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
# 크롤링 잘 됐는지 확인
df['content']

1    내년이면 서른이다.\n벌써 93년생이 서른이 되는 해가 온 것이다.\n하지만 서른이...
4    1월 회계학원 시작, 셒구워크숍\n2월 회계학원을 계속 다님. 컴활1급 계속 떨어지...
7                                                     
8    벌써 12월이라는데 믿을 수가 업써...\n12월 목표 소비 줄이기...\n특히 옷...
9    *욕밖에 없으니 긍정적으로 한 해를 마무리 하고 싶은 사람은 보지 않는 걸 추천함\...
Name: content, dtype: object